# BASIN_HOPPING

## Overview
This function provides a robust global optimization method using the **basinhopping algorithm** from SciPy. Basinhopping is a stochastic global optimization technique that combines random perturbations with local minimization to escape local minima and search for the global minimum of a scalar function. It is especially useful for non-convex optimization problems where the objective function may have multiple local minima or complex landscapes.

The basinhopping algorithm seeks to solve $\min_x f(x)$, where $f(x)$ is the objective function. At each iteration, the algorithm performs a random displacement $x_{\text{trial}} = x_{\text{current}} + \delta$, where $\delta$ is a random vector (with user-defined step size). A local minimization is then performed starting from $x_{\text{trial}}$ to find $x_{\text{min}} = \operatorname{argmin}_x f(x)$. The new minimum is accepted with probability $P = 1$ if $\Delta f \leq 0$, or $P = \exp(-\Delta f / T)$ if $\Delta f > 0$, where $\Delta f = f(x_{\text{min}}) - f(x_{\text{current}})$ and $T$ is the temperature parameter. This approach allows the algorithm to escape local minima by occasionally accepting worse solutions, similar to simulated annealing, and increases the chance of finding the global minimum.

**Key Features:**
- Supports both univariate and multivariate functions (with initial guess as a scalar or 2D list).
- Accepts user-defined Python lambda expressions for the objective function, compatible with NumPy and math operations.
- Allows customization of algorithm parameters: number of iterations, temperature, step size, and local minimizer method.
- Returns the minimum value found and the corresponding location in the variable space.

**Applications:**
- Engineering design optimization
- Financial modeling and portfolio optimization
- Scientific computing and parameter estimation
- Any scenario where the objective function is non-convex or has multiple local minima

Basinhopping is more robust than local optimizers for difficult landscapes and is widely used in research and industry for global optimization tasks.

## Usage
To use the `basin_hopping` function in Excel, enter it as a formula in a cell, specifying the function to minimize, the initial guess, and any optional arguments:

```excel
=BASIN_HOPPING(func_expr, x_zero, [niter], [T], [stepsize], [minimizer_method])
```
- `func_expr` should be a string representing a valid Python lambda expression of one variable, e.g., "lambda x: x**2 + 10*sin(x)".
- `x_zero` is the initial guess (scalar or 2D list, e.g., [[0, 0]] for two variables).

## Arguments
| Argument         | Type         | Required | Description                                                                 | Example |
|-------------------|--------------|----------|-----------------------------------------------------------------------------|---------|
| func_expr         | string       | Yes      | Python lambda expression as a string                                        | "lambda x: (x-3)**2 + 2" |
| x_zero            | scalar/2D list | Yes      | Initial guess for the variables (scalar or 2D list)                         | 0 or [[0, 0]] |
| niter             | int          | No       | Number of basinhopping iterations (default: 100)                            | 100 |
| T                 | float        | No       | Temperature parameter for acceptance test (default: 1.0)                    | 1.0 |
| stepsize          | float        | No       | Step size for random displacement (default: 0.5)                            | 0.5 |
| minimizer_method  | string       | No       | Local minimization method (default: 'L-BFGS-B')                             | "L-BFGS-B" |

## Returns
| Returns     | Type   | Description                                  | Example |
|--------------|--------|----------------------------------------------|---------|
| Minimum      | float  | The minimum value found by the algorithm      | 2.0 |
| x_min        | list   | The location (as a list) of the minimum      | [3.0] or [1.0, -2.0] |

Returns a 2D list: [[minimum, x_min[0], x_min[1], ...]]

## Examples

### Minimize a simple quadratic
**Input:**
| func_expr                  | x_zero |
|---------------------------|--------|
| "lambda x: (x-3)**2 + 2"  | 0      |

**Call:**
```excel
=BASIN_HOPPING("lambda x: (x-3)**2 + 2", 0)
```
**Output:**
| Minimum | x_min |
|---------|-------|
| 2.0     | 3.0   |

### Minimize a multimodal function
**Input:**
| func_expr                        | x_zero |
|----------------------------------|--------|
| "lambda x: x**2 + 10*sin(x)"     | 2      |

**Call:**
```excel
=BASIN_HOPPING("lambda x: x**2 + 10*sin(x)", 2)
```
**Output:**
| Minimum   | x_min   |
|-----------|---------|
| ~-7.945   | ~-1.427 |

### Multivariate function
**Input:**
| func_expr                                 | x_zero   |
|--------------------------------------------|----------|
| "lambda x: (x[0]-1)**2 + (x[1]+2)**2"     | {{0,0}}  |

**Call:**
```excel
=BASIN_HOPPING("lambda x: (x[0]-1)**2 + (x[1]+2)**2", {{0,0}})
```
**Output:**
| Minimum | x0_min | x1_min |
|---------|--------|--------|
| 0.0     | 1.0    | -2.0   |

In [ ]:
import numpy as np
from scipy.optimize import basinhopping as scipy_basinhopping
import math

def basin_hopping(func_expr, x_zero, niter=100, T=1.0, stepsize=0.5, minimizer_method='L-BFGS-B'):
    """
    Finds the global minimum of a function using the basinhopping algorithm.

    Args:
        func_expr (str): Python lambda expression as a string, e.g., "lambda x: x**2 + 10*math.sin(x)"
        x_zero (scalar or 2D list): Initial guess. Must be a scalar or a 2D list (e.g., [[0, 0]] for 2 variables)
        niter (int, optional): Number of basinhopping iterations
        T (float, optional): Temperature parameter
        stepsize (float, optional): Step size for random displacement
        minimizer_method (str, optional): Local minimization method

    Returns:
        2D list: [[minimum, x_min[0], x_min[1], ...]] or [[error_message]]
    """
    try:
        # Validate and parse func_expr
        if not isinstance(func_expr, str):
            return [["func_expr must be a string representing a lambda expression."]]
        try:
            user_func = eval(func_expr, {"np": np, "math": math, "sin": math.sin, "cos": math.cos, "exp": math.exp, "sqrt": math.sqrt})
        except Exception as e:
            return [[f"Invalid function expression: {str(e)}"]]
        # Wrap user_func to always return a scalar float
        def func(x):
            try:
                val = user_func(x)
                if isinstance(val, np.ndarray):
                    return float(val.item())
                return float(val)
            except Exception as e:
                return float('inf')  # Penalize errors in user function
        # Validate x_zero
        if isinstance(x_zero, (int, float)):
            x0_val = float(x_zero)
        elif isinstance(x_zero, (list, tuple, np.ndarray)):
            arr = np.array(x_zero, dtype=float)
            if arr.ndim == 2 and arr.shape[0] == 1:
                x0_val = arr[0]
            else:
                return [["x_zero must be a scalar or a 2D list with shape (1, n)."]]
        else:
            return [["x_zero must be a scalar or a 2D list."]]
        if not isinstance(niter, int) or niter <= 0:
            return [["niter must be a positive integer."]]
        if not isinstance(T, (int, float)) or T <= 0:
            return [["T must be a positive number."]]
        if not isinstance(stepsize, (int, float)) or stepsize <= 0:
            return [["stepsize must be a positive number."]]
        if not isinstance(minimizer_method, str):
            return [["minimizer_method must be a string."]]
        result = scipy_basinhopping(func, x0_val, niter=niter, T=T, stepsize=stepsize, minimizer_kwargs={"method": minimizer_method})
        x_min = result.x
        min_val = result.fun
        if isinstance(min_val, np.ndarray):
            min_val = float(min_val.item())
        else:
            min_val = float(min_val)
        if isinstance(x_min, np.ndarray):
            x_min_list = x_min.tolist()
        else:
            x_min_list = [float(x_min)]
        return [[min_val] + x_min_list]
    except Exception as e:
        return [[str(e)]]

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()

def test_quadratic():
    result = basin_hopping("lambda x: (x-3)**2 + 2", 0)
    assert isinstance(result, list)
    assert len(result) == 1
    assert abs(result[0][0] - 2.0) < 0.05
    assert abs(result[0][1] - 3.0) < 0.05

def test_multimodal():
    result = basin_hopping("lambda x: x**2 + 10*math.sin(x)", -2, niter=300)
    assert isinstance(result, list)
    assert len(result) == 1
    assert -8.1 <= result[0][0] <= -7.3
    assert -1.7 <= result[0][1] <= -1.3

def test_multivariate():
    result = basin_hopping("lambda x: (x[0]-1)**2 + (x[1]+2)**2", [[0, 0]], niter=200)
    assert isinstance(result, list)
    assert len(result) == 1
    assert abs(result[0][0]) < 0.05
    assert 0.99 <= result[0][1] <= 1.01
    assert -2.01 <= result[0][2] <= -1.99

ipytest.run()

In [ ]:
# Interactive Demo
import gradio as gr

examples = [
    ["lambda x: (x-3)**2 + 2", "0", 100, 1.0, 0.5, "L-BFGS-B"],
    ["lambda x: x**2 + 10*math.sin(x)", "-2", 300, 1.0, 0.5, "L-BFGS-B"],
    ["lambda x: (x[0]-1)**2 + (x[1]+2)**2", "[[0, 0]]", 200, 1.0, 0.5, "L-BFGS-B"]
]

demo = gr.Interface(
    fn=basin_hopping,
    inputs=[
        gr.Textbox(label="Function Expression", lines=2, value="lambda x: (x-3)**2 + 2"),
        gr.DataFrame(label="Initial Guess (x_zero)", value=[[0]], type="array", datatype="number", column_widths=['20%']),
        gr.Number(label="Number of Iterations (niter)", value=100),
        gr.Number(label="Temperature (T)", value=1.0),
        gr.Number(label="Step Size", value=0.5),
        gr.Textbox(label="Minimizer Method", value="L-BFGS-B"),
    ],
    outputs=gr.Dataframe(label="Result Table or Error Message"),
    examples=[
        ["lambda x: (x-3)**2 + 2", [[0]], 100, 1.0, 0.5, "L-BFGS-B"],
        ["lambda x: x**2 + 10*math.sin(x)", [[-2]], 300, 1.0, 0.5, "L-BFGS-B"],
        ["lambda x: (x[0]-1)**2 + (x[1]+2)**2", [[0, 0]], 200, 1.0, 0.5, "L-BFGS-B"]
    ],
    description="Find the global minimum of a function using the basin_hopping algorithm.",
    flagging_mode="never",
)
demo.launch()